<a href="https://colab.research.google.com/github/zacherymoy/DS-Unit-4-Sprint-2-Neural-Networks/blob/master/Copy_of_LS_DS_434_Hyperparameter_Tuning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [11]:
!pip install keras-tuner


     |████████████████████████████████| 61kB 2.6MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=29996b6335b816e0256214ae11cff8b324d9e5b4f2abd86120e20f5dc5731c20
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=c98c7cab69370d65ae6eaf2282287f8a07aa4a376e84510acb172f65e0e3fb9e
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


## Cleaning 

In [0]:
# Import packages
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np

In [0]:
df = pd.read_csv('https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv')

In [0]:
# check out what I need to clean. Need headers 
df.head()

In [17]:
# standardize column names 
df.columns = [col.lower() for col in df.columns]
df = df.drop(columns='customerid')
df['totalcharges'] = df['totalcharges'].map({' ':0.0}, na_action='ignore')

KeyError: ignored

In [0]:
# set up target and features
target = 'churn'
non_cat_feats = ['seniorcitizen','tenure','monthlycharges','totalcharges']
cat_feats = list(df.drop(columns=[target]+non_cat_feats).columns)
features = cat_feats + non_cat_feats

In [19]:
# Good! 
df.head()

,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,NaN,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,NaN,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,NaN,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,NaN,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,NaN,Yes


## Create and fit a baseline Keras MLP model to the data.

In [20]:
label = LabelEncoder()
label.fit(df[target])

LabelEncoder()

In [21]:
ordinal = OrdinalEncoder()
ordinal.fit(df[cat_feats])

OrdinalEncoder(categories='auto', dtype=<class 'numpy.float64'>)

In [0]:
def preprocess(df):
  '''
  Preprocessing function

  inputs: DataFrame

  outputs: X_train, y_train as numpy arrays
  '''
  # copy from original dataframe
  X = df[features].copy()
  y = df[target].copy()

  # transform target
  y = label.transform(y)

  # transform categoricals
  X_cat = ordinal.transform(X[cat_feats])
  # put transformed categoricals back into dataframe to maintain structure
  X_cat = pd.DataFrame(X_cat, columns=cat_feats)
  for col in cat_feats:
    X[col] = X_cat[col]
  for col in X.columns:
    X[col] = pd.to_numeric(X[col])
  # print(X.dtypes)
  # print(X['totalcharges'][:5])
  return np.asarray(X.values).astype(np.float32), np.asarray(y).astype(np.float32)

In [0]:
X_train, y_train = preprocess(df)


In [24]:
def create_model(layers=0, num_nodes=32, act_type='relu'):
  model = keras.models.Sequential()
  model.add(keras.layers.Dense(num_nodes, activation=act_type))
  for i in range(layers):
    model.add(keras.layers.Dense(num_nodes, activation=act_type))
  model.add(keras.layers.Dense(2, activation='softmax'))
  model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])
  return model

model = KerasClassifier(build_fn=create_model, verbose=0)

param_grid = {'layers': [0, 1, 2, 3],
              'num_nodes': [8, 16, 32, 64, 128],
              'act_type': ['relu','sigmoid','selu','tanh']}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, verbose=2)
grid_result = grid.fit(X_train, y_train)
print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Fitting 5 folds for each of 80 candidates, totalling 400 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] act_type=relu, layers=0, num_nodes=8 ............................
[CV] ............. act_type=relu, layers=0, num_nodes=8, total=   1.5s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.5s remaining:    0.0s


[CV] act_type=relu, layers=0, num_nodes=8 ............................
[CV] ............. act_type=relu, layers=0, num_nodes=8, total=   0.9s
[CV] act_type=relu, layers=0, num_nodes=8 ............................
[CV] ............. act_type=relu, layers=0, num_nodes=8, total=   0.9s
[CV] act_type=relu, layers=0, num_nodes=8 ............................
[CV] ............. act_type=relu, layers=0, num_nodes=8, total=   0.9s
[CV] act_type=relu, layers=0, num_nodes=8 ............................
[CV] ............. act_type=relu, layers=0, num_nodes=8, total=   1.1s
[CV] act_type=relu, layers=0, num_nodes=16 ...........................
[CV] ............ act_type=relu, layers=0, num_nodes=16, total=   0.9s
[CV] act_type=relu, layers=0, num_nodes=16 ...........................
[CV] ............ act_type=relu, layers=0, num_nodes=16, total=   0.9s
[CV] act_type=relu, layers=0, num_nodes=16 ...........................
[CV] ............ act_type=relu, layers=0, num_nodes=16, total=   0.9s
[CV] a

[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed:  7.8min finished


Best: 0.7346286535263061 using {'act_type': 'relu', 'layers': 0, 'num_nodes': 8}
Means: 0.7346286535263061, Stdev: 0.004757632470334884 with: {'act_type': 'relu', 'layers': 0, 'num_nodes': 8}
Means: 0.7346286535263061, Stdev: 0.004757632470334884 with: {'act_type': 'relu', 'layers': 0, 'num_nodes': 16}
Means: 0.7346286535263061, Stdev: 0.004757632470334884 with: {'act_type': 'relu', 'layers': 0, 'num_nodes': 32}
Means: 0.7346286535263061, Stdev: 0.004757632470334884 with: {'act_type': 'relu', 'layers': 0, 'num_nodes': 64}
Means: 0.7346286535263061, Stdev: 0.004757632470334884 with: {'act_type': 'relu', 'layers': 0, 'num_nodes': 128}
Means: 0.7346286535263061, Stdev: 0.004757632470334884 with: {'act_type': 'relu', 'layers': 1, 'num_nodes': 8}
Means: 0.7346286535263061, Stdev: 0.004757632470334884 with: {'act_type': 'relu', 'layers': 1, 'num_nodes': 16}
Means: 0.7346286535263061, Stdev: 0.004757632470334884 with: {'act_type': 'relu', 'layers': 1, 'num_nodes': 32}
Means: 0.734628653526306

## Past tries

In [0]:
# import csv
# with open('WA_Fn-UseC_-Telco-Customer-Churn+(1).csv', 'r',) as file:
#     reader = csv.reader(file, delimiter = '\t')
#     for row in reader:
#         print(row)

In [0]:
# import csv

# with open('WA_Fn-UseC_-Telco-Customer-Churn+(1).csv', 'w', newline='') as file:
#     fieldnames = ['player_name', 'fide_rating']
#     writer = csv.DictWriter(file, fieldnames=fieldnames)

#     writer.writeheader()
#     writer.writerow({'player_name': 'Magnus Carlsen', 'fide_rating': 2870})
#     writer.writerow({'player_name': 'Fabiano Caruana', 'fide_rating': 2822})
#     writer.writerow({'player_name': 'Ding Liren', 'fide_rating': 2801})

In [7]:
# import csv

# FILE_NAME = "WA_Fn-UseC_-Telco-Customer-Churn+(1).csv"
# COL_HEADERS = ['customer ID', 'gender', 'senior c', 'partner', 'depend', 'tenure', 'Phone service', 'Multiple lines',
#                'Internet service', 'Oneline Security', 'Online Backup', 'Device Protection', 'Tech Support',
#                'Streaming TV', 'Streaming Movies', 'Contract', 'Paperless Billing', 'Payment Method', 'Monthly Charges',
#                'Total Charges', 'Churn']
# NUM_COLS = len(COL_HEADERS)

# # read file once to determine maximum width of data in columns
# with open(FILE_NAME) as f:
#     reader = csv.reader(f, delimiter=',')
#     # determine the maximum width of the data in each column
#     max_col_widths = [len(col_header) for col_header in COL_HEADERS]
#     for columns in reader:
#         if "A" in columns and int(columns[5]) < int(columns[3]):
#             for i, col in enumerate(columns):
#                     max_col_widths[i] = max(max_col_widths[i], len(repr(col)))
#     # add 1 to each for commas
#     max_col_widths = [col_width+1 for col_width in max_col_widths]

# # read file second time to display its contents with the headers
# with open(FILE_NAME) as f:
#     reader = csv.reader(f, delimiter=',')
#     # display justified column headers
#     print(' ' + ' '.join(col_header.ljust(max_col_widths[i])
#                             for i, col_header in enumerate(COL_HEADERS)))
#     # display column data
#     for columns in reader:
#         if "A" in columns and int(columns[5]) < int(columns[3]):
#             print(columns)

 customer ID  gender  senior c  partner  depend  tenure  Phone service  Multiple lines  Internet service  Oneline Security  Online Backup  Device Protection  Tech Support  Streaming TV  Streaming Movies  Contract  Paperless Billing  Payment Method  Monthly Charges  Total Charges  Churn 


In [10]:
# import csv

# FILE_NAME = "WA_Fn-UseC_-Telco-Customer-Churn+(1).csv"
# COL_HEADERS = ['customer ID', 'gender', 'senior c', 'partner', 'depend', 'tenure', 'Phone service', 'Multiple lines',
#                'Internet service', 'Oneline Security', 'Online Backup', 'Device Protection', 'Tech Support',
#                'Streaming TV', 'Streaming Movies', 'Contract', 'Paperless Billing', 'Payment Method', 'Monthly Charges',
#                'Total Charges', 'Churn']
# NUM_COLS = len(COL_HEADERS)

# with open(FILE_NAME) as file:
#     reader = csv.reader(FILE_NAME)
#     for row in reader:
#         print(row)

['W']
['A']
['_']
['F']
['n']
['-']
['U']
['s']
['e']
['C']
['_']
['-']
['T']
['e']
['l']
['c']
['o']
['-']
['C']
['u']
['s']
['t']
['o']
['m']
['e']
['r']
['-']
['C']
['h']
['u']
['r']
['n']
['+']
['(']
['1']
[')']
['.']
['c']
['s']
['v']


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?